In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from transformers import BertTokenizer, BertModel
from datasets import load_dataset
import torch

import re
import torch
from transformers import BertTokenizer, BertModel
from datasets import load_dataset

##Carga del dataset
Dataset: https://huggingface.co/datasets/Emilianohack6950/Wikipedia-es

In [ ]:
#Cargamos el dataset
ds = load_dataset("Emilianohack6950/Wikipedia-es")
ds

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/800 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'título', 'contenido'],
        num_rows: 800
    })
    test: Dataset({
        features: ['id', 'título', 'contenido'],
        num_rows: 200
    })
})

In [ ]:
train_contenido = [item["contenido"] for item in ds["train"]]
test_contenido = [item["contenido"] for item in ds["test"]]

todo_contenido = train_contenido + test_contenido

# Unir todo en una sola cadena
texto_total = " ".join(todo_contenido)

# Contar signos de puntuación
cant_comas = texto_total.count(',')
cant_puntos = texto_total.count('.')
cant_preg_abre = texto_total.count('¿')
cant_preg_cierra = texto_total.count('?')

# Contar letras mayúsculas (A-Z, incluyendo acentos)
cant_mayus = len(re.findall(r'[A-ZÁÉÍÓÚÜÑ]', texto_total))

print("Estadísticas del dataset:")
print(f"Comas (,): {cant_comas:,}")
print(f"Puntos (.): {cant_puntos:,}")
print(f"Signos de pregunta abiertos (¿): {cant_preg_abre:,}")
print(f"Signos de pregunta cerrados (?): {cant_preg_cierra:,}")
print(f"Letras mayúsculas: {cant_mayus:,}")

Estadísticas del dataset:
Comas (,): 22,160
Puntos (.): 16,364
Signos de pregunta abiertos (¿): 21
Signos de pregunta cerrados (?): 53
Letras mayúsculas: 72,640


## Funciones de preprocesamiento

In [ ]:
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
model.eval()

# Funcion provista por la catedra
def get_multilingual_token_embedding(token: str):
  """     Devuelve el embedding (estático) para el token.     """
  token_id = tokenizer.convert_tokens_to_ids(token)
  if token_id is None or token_id == tokenizer.unk_token_id:
    print(f" ❌ El token '{token}' no pertenece al vocabulario de multilingual BERT.")
    return None
  embedding_vector = model.embeddings.word_embeddings.weight[token_id]
  print(f" ✅ Token: '{token}' | ID: {token_id}")
  print(f"Embedding shape: {embedding_vector.shape}")
  return embedding_vector



def tipo_capitalizacion(palabra):
    conteo_mayusculas = 0
    for caracter in palabra:
        if caracter.isupper():
            conteo_mayusculas += 1

    if palabra.isupper():
        return 3
    else:
        if conteo_mayusculas >= 2:
            return 2
        else:
            if conteo_mayusculas == 1:
                if palabra[0].isupper():
                    return 1
                else:
                    return 2
            else:
                return 0

#Capaz conviene tokenizar todo de una en vez de palabra por palabra

def tokenizar_y_etiquetar(texto_original, instancia_id = 1):
  #Agrupamos el texto original en palabras y signo por ej:
  # "¿Cuándo vamos a McDonald's?"
  # ["¿Cuándo ", "vamos", "a", "McDonald's?"]
  # Para cada palabra y signo, aplanamos el texto y lo tokenizamos
  #        ORIGINAL              PLANO TOKENIZADO
  #     ["¿Cuándo "]    -> ["cu", "##ánd", "##o"]
  #     ["vamos"]       -> ["va", "##mos"]
  #     ["a"]           -> ["a"]
  #     ["McDonald's?"] -> ["m", "##c", "##dona", "##d", "'","s"]
  # Ya tenemos toda la imformacón para armar las etiquetas:
  #  input  -> token
  #  output -> [punt_inicial,punt_final,capitalización]
  #

  SIGNOS_ESPANOL = "¿¡?.,!\"'’“”()[]:;—-"

  #Separamos por palabra y signo
  partes = re.findall(r"\w+['’]?\w*|[¿?.,]", texto_original)
  resultado = []

  #Para cada Palabra y Signo, aplanamos y tokenizamos
  #Extraemos Capitalización y puntuación de la palabra y signos adyacentes, y se la asignamos a los tokens
  for i in range(len(partes)):
    parte = partes[i]

    if(parte not in  SIGNOS_ESPANOL):
      #tokens = tokenizer.tokenize(parte.lower().strip("¿?.,¡!"))
      tokens = tokenizer.tokenize(parte.lower()) # Sacamos mayúsculas y la tokenizamos
      #Obtenemos la capitalización
      cap = tipo_capitalizacion(parte)

      #Obtenemos la puntuación por token
      punt_ini = '¿' if partes[i-1]=='¿' else ''
      punt_fin = ''
      if(i<len(partes)-1):
        if partes[i+1] in '.?,':
          punt_fin = partes[i+1]

      for j in range(len(tokens)):
        tok = tokens[j]
        punt_ini_token = ''
        punt_fin_token = ''
        if(j==0):#Si es el primer token se le agrega puntuación inicial ¿
            punt_ini_token = punt_ini
        if (j==len(tokens)-1): #Si el último token se le agrega puntuación final
            punt_fin_token = punt_fin

        token_id = tokenizer.convert_tokens_to_ids(tok)
        resultado.append({
                    "instancia_id": instancia_id,
                    "token_id": token_id,
                    "token": tok,
                    "punt_inicial": punt_ini_token,
                    "punt_final": punt_fin_token,
                    "capitalizacion": cap
                })

  return resultado

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

## Testing para tokenizar y etiquetar

In [ ]:
import unittest

class TestFunciones(unittest.TestCase):

    def simplificar_salida(self, salida):
        return [
            [item["token"], item["punt_inicial"], item["punt_final"], item["capitalizacion"]]
            for item in salida
        ]

    def test01_reconoce_Capitalizacion_Token_inicial_mas_de_una_palabra(self):
        texto_1 = "Los árboles altos"
        salida = self.simplificar_salida(tokenizar_y_etiquetar(texto_1))
        self.assertEqual(
            salida,
            [['los','','',1], ['árboles','','',0], ['altos','','',0]]
        )

    def test02_reconoce_Punto_final_mas_de_una_palabra(self):
        texto_1 = "Los árboles altos."
        salida = self.simplificar_salida(tokenizar_y_etiquetar(texto_1))
        self.assertEqual(
            salida,
            [['los','','',1], ['árboles','','',0], ['altos','','.',0]]
        )

    def test03_reconoce_Capitalizacion_Todos_los_tokens(self):
        texto_1 = "Hermosos son los árboles altos."
        salida = self.simplificar_salida(tokenizar_y_etiquetar(texto_1))
        self.assertEqual(
            salida,
            [['her','','',1], ['##mos','','',1], ['##os','','',1],
             ['son','','',0], ['los','','',0], ['árboles','','',0], ['altos','','.',0]]
        )

    def test04_reconoce_Capitalizacion_todos_los_tokens_May_Min(self):
        texto_1 = "Hermosos son los árboles hermosos."
        salida = self.simplificar_salida(tokenizar_y_etiquetar(texto_1))
        self.assertEqual(
            salida,
            [['her','','',1], ['##mos','','',1], ['##os','','',1],
             ['son','','',0], ['los','','',0], ['árboles','','',0],
             ['her','','',0], ['##mos','','',0], ['##os','','.',0]]
        )

    def test05_reconoce_capitalizacion_Signo_pregunta_punto(self):
        texto_1 = "¿Cuándo vamos a McDonald's? Ellos no vienen hoy. ¿Dónde están ahora?"
        salida = self.simplificar_salida(tokenizar_y_etiquetar(texto_1))
        self.assertEqual(
            salida,
            [['cu','¿','',1], ['##ánd','','',1], ['##o','','',1], ['va','','',0],
             ['##mos','','',0], ['a','','',0], ['m','','',2], ['##c','','',2],
             ['##dona','','',2], ['##ld','','',2], ["'",'','',2], ['s','','?',2],
             ['ellos','','',1], ['no','','',0], ['viene','','',0], ['##n','','',0],
             ['hoy','','.',0], ['dó','¿','',1], ['##nde','','',1],
             ['están','','',0], ['ahora','','?',0]]
        )

    def test06_reconoce_comas(self):
        texto_1 = "Si, es correcto."
        salida = self.simplificar_salida(tokenizar_y_etiquetar(texto_1))
        self.assertEqual(
            salida,
            [['si','',',',1], ['es','','',0], ['correct','','',0], ['##o','','.',0]]
        )

    def test07_reconoce_varias_comas(self):
        texto = "Sí, claro, entiendo."
        salida = self.simplificar_salida(tokenizar_y_etiquetar(texto))
        self.assertEqual(
            salida,
            [['sí','',',',1], ['claro','',',',0], ['ent','','',0], ['##iendo','','.',0]]
        )

    def test08_reconoce_todas_mayusculas(self):
        texto = "Me gustaría ir a la NASA."
        salida = self.simplificar_salida(tokenizar_y_etiquetar(texto))
        self.assertEqual(
            salida,
            [['me','','',1], ['gu','','',0], ['##star','','',0], ['##ía','','',0],
             ['ir','','',0], ['a','','',0], ['la','','',0], ['nasa','','.',3]]
        )

    def test09_reconoce_algunas_mayusculas(self):
        texto = "El iPhone es caro."
        salida = self.simplificar_salida(tokenizar_y_etiquetar(texto))
        self.assertEqual(
            salida,
            [['el','','',1], ['i','','',2], ['##phone','','',2], ['es','','',0],
             ['car','','',0], ['##o','','.',0]]
        )

    def test10_otros_signos_no_afectan(self):
        texto = "¡Hola Mundo!(a)[A];a:A-a"
        salida = self.simplificar_salida(tokenizar_y_etiquetar(texto))
        self.assertEqual(
            salida,
            [['hol','','',1], ['##a','','',1], ['mundo','','',1],
             ['a','','',0], ['a','','',3], ['a','','',0],
             ['a','','',3], ['a','','',0]]
        )


if __name__ == "__main__":
    unittest.main(argv=[''], verbosity=2, exit=False)

test01_reconoce_Capitalizacion_Token_inicial_mas_de_una_palabra (__main__.TestFunciones.test01_reconoce_Capitalizacion_Token_inicial_mas_de_una_palabra) ... ok
test02_reconoce_Punto_final_mas_de_una_palabra (__main__.TestFunciones.test02_reconoce_Punto_final_mas_de_una_palabra) ... ok
test03_reconoce_Capitalizacion_Todos_los_tokens (__main__.TestFunciones.test03_reconoce_Capitalizacion_Todos_los_tokens) ... ok
test04_reconoce_Capitalizacion_todos_los_tokens_May_Min (__main__.TestFunciones.test04_reconoce_Capitalizacion_todos_los_tokens_May_Min) ... ok
test05_reconoce_capitalizacion_Signo_pregunta_punto (__main__.TestFunciones.test05_reconoce_capitalizacion_Signo_pregunta_punto) ... ok
test06_reconoce_comas (__main__.TestFunciones.test06_reconoce_comas) ... ok
test07_reconoce_varias_comas (__main__.TestFunciones.test07_reconoce_varias_comas) ... ok
test08_reconoce_todas_mayusculas (__main__.TestFunciones.test08_reconoce_todas_mayusculas) ... ok
test09_reconoce_algunas_mayusculas (__main

In [ ]:
texto_1 = "¿Cuándo vamos a McDonald's? Ellos no vienen hoy. ¿Dónde están ahora?"
texto_2 = "cuándo vamos a mcdonald's ellos no vienen hoy dónde están ahora"
ej1_tokens = tokenizer.tokenize(texto_1)
ej2_tokens = tokenizer.tokenize(texto_2)
print(f"Tokens original: {ej1_tokens}")
print(f"Tokens plano: {ej2_tokens}")



Tokens original: ['¿', 'Cu', '##ánd', '##o', 'va', '##mos', 'a', 'McDonald', "'", 's', '?', 'El', '##los', 'no', 'viene', '##n', 'hoy', '.', '¿', 'D', '##ón', '##de', 'están', 'ahora', '?']
Tokens plano: ['cu', '##ánd', '##o', 'va', '##mos', 'a', 'm', '##c', '##dona', '##ld', "'", 's', 'ellos', 'no', 'viene', '##n', 'hoy', 'dó', '##nde', 'están', 'ahora']


## Conversion en embeddings y tensores.

In [ ]:
MAP_PUNT_INI = {"": 0, "¿": 1}
MAP_PUNT_FIN = {"": 0, ",": 1, ".": 2, "?": 3}

# Faltaría usar dataset y data loader de pytorch

def procesar_texto_a_tensores(texto, instancia_id=1):
    etiquetas = tokenizar_y_etiquetar(texto, instancia_id)

    tokens = [fila["token"] for fila in etiquetas]

    lista_embeddings = []
    for tok in tokens:
        emb = get_multilingual_token_embedding(tok)
        lista_embeddings.append(emb)

    embeddings_tensor = torch.stack(lista_embeddings)

    # Outputs (convertidos a tensores)
    y_cap = torch.tensor([fila["capitalizacion"] for fila in etiquetas])
    y_punt_ini = torch.tensor([MAP_PUNT_INI[fila["punt_inicial"]] for fila in etiquetas])
    y_punt_fin = torch.tensor([MAP_PUNT_FIN[fila["punt_final"]] for fila in etiquetas])


    return {
        "instancia_id": instancia_id,
        "tokens": tokens,
        "X_emb": embeddings_tensor,
        "y_cap": y_cap,
        "y_punt_ini": y_punt_ini,
        "y_punt_fin": y_punt_fin,
        "etiquetas": etiquetas
    }

In [ ]:
texto = "¿Cuándo vamos a McDonald's? Ellos no vienen hoy. ¿Dónde están ahora?"
datos = procesar_texto_a_tensores(texto, instancia_id=1)

print("Tokens:", datos["tokens"])
print("Embeddings shape:", datos["X_emb"].shape)
print("Capitalización:", datos["y_cap"])
print("Puntuación inicial:", datos["y_punt_ini"])
print("Puntuación final:", datos["y_punt_fin"])

 ✅ Token: 'cu' | ID: 10854
Embedding shape: torch.Size([768])
 ✅ Token: '##ánd' | ID: 101439
Embedding shape: torch.Size([768])
 ✅ Token: '##o' | ID: 10133
Embedding shape: torch.Size([768])
 ✅ Token: 'va' | ID: 10321
Embedding shape: torch.Size([768])
 ✅ Token: '##mos' | ID: 13386
Embedding shape: torch.Size([768])
 ✅ Token: 'a' | ID: 169
Embedding shape: torch.Size([768])
 ✅ Token: 'm' | ID: 181
Embedding shape: torch.Size([768])
 ✅ Token: '##c' | ID: 10350
Embedding shape: torch.Size([768])
 ✅ Token: '##dona' | ID: 64674
Embedding shape: torch.Size([768])
 ✅ Token: '##ld' | ID: 12620
Embedding shape: torch.Size([768])
 ✅ Token: ''' | ID: 112
Embedding shape: torch.Size([768])
 ✅ Token: 's' | ID: 187
Embedding shape: torch.Size([768])
 ✅ Token: 'ellos' | ID: 16169
Embedding shape: torch.Size([768])
 ✅ Token: 'no' | ID: 10192
Embedding shape: torch.Size([768])
 ✅ Token: 'viene' | ID: 12266
Embedding shape: torch.Size([768])
 ✅ Token: '##n' | ID: 10115
Embedding shape: torch.Size([768]